# Descripción del proyecto
Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
Crear un modelo para predecir el volumen de reservas en pozos nuevos.
Elegir los pozos petrolíferos que tienen los valores estimados más altos.
Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.
Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

# Descarga y prepara los datos

In [1]:
# Importar librerias 
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# Carga del datasets
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# Analisis de df_0
df_0.info() # informacion general 
print(df_0.head()) # imprimir las primeras filas
print('Numero de valores ausentes',df_0.isna().sum()) # revisar si  hay valores ausentes
print('Numero de registros duplicados',df_0.duplicated().sum()) # revisar si hay filas duplicadas 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
Numero de valores ausentes id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Numero de registros duplicados 0


In [4]:
# Analisis de df_1
df_1.info() # informacion general 
print(df_1.head()) # imprimir las primeras filas
print('Numero de valores ausentes',df_1.isna().sum()) # revisar si  hay valores ausentes
print('Numero de registros duplicados',df_1.duplicated().sum()) # revisar si hay filas duplicadas 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
Numero de valores ausentes id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Numero de registros duplicados 0


In [5]:
# Analisis de df_2
df_2.info() # informacion general 
print(df_2.head()) # imprimir las primeras filas
print('Numero de valores ausentes',df_2.isna().sum()) # revisar si  hay valores ausentes
print('Numero de registros duplicados',df_2.duplicated().sum()) # revisar si hay filas duplicadas 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746
Numero de valores ausentes id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Numero de registros duplicados 0


In [6]:
# División del dataset en features y targets
features_0 = df_0.drop(['id','product'], axis=1)
features_1 = df_1.drop(['id','product'], axis=1)
features_2 = df_2.drop(['id','product'], axis=1)
target_0 = df_0['product']
target_1 = df_1['product']
target_2 = df_2['product']

In [7]:
# División del dataset en entrenamiento y validación
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)
features_train_0, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)
features_train_0, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

Los tres dataframes tienen 100000 registros con 5 columnas cada uno, no tiene valores ausentes ni duplicados
Además, las caracteristica f0, f1 y f2, son significativas pues no se deben de escalar entoces los datos estan listo para crear el modelo, la columna de ids, no sera considera ya que no aportar informacion validosa.

# Entrena y prueba el modelo

In [8]:
# Creamos una funcion para realizar el modelo de regresion lineal
def Modelo_reg (features, target, features_valid, target_valid):
    model = LinearRegression() # Creamos el modelo
    model.fit(features, target) # Entrenamos el modelo
    predictions = model.predict(features_valid) # Guardamos las prediciones
    mean_pred = np.mean(predictions) # Calculamos la media de las prediciones
    mean_real = np.mean(target_valid) # Calculamos la media de los valores realeas
    rmse = np.sqrt(mean_squared_error(target_valid, predictions)) # Calcula el RMSE
    print('La media de las predicciones es:', mean_pred) # Imprimos la media de las predicciones
    print('La media de los valores reales es:', mean_real) # Imprimos la media de los valores reales
    print('El RMSE del modelo es:', rmse) # Imprimos el RMSE
    
    return mean_pred, mean_real, rmse, predictions

In [9]:
# Valores de df_0
mean_pred_0, mean_real_0, rmse_0, predictions_0 = Modelo_reg (features_0, target_0, features_valid_0, target_valid_0)

La media de las predicciones es: 92.46564781325566
La media de los valores reales es: 92.07859674082927
El RMSE del modelo es: 37.57547919032473


In [10]:
# Valores de df_1
mean_pred_1, mean_real_1, rmse_1, predictions_1 = Modelo_reg (features_1, target_1, features_valid_1, target_valid_1)

La media de las predicciones es: 68.72718652381815
La media de los valores reales es: 68.72313602435997
El RMSE del modelo es: 0.8930685055287833


In [11]:
# Valores de df_2
mean_pred_2, mean_real_2, rmse_2, predictions_2 = Modelo_reg (features_2, target_2, features_valid_2, target_valid_2)

La media de las predicciones es: 94.94513027263153
La media de los valores reales es: 94.88423280885438
El RMSE del modelo es: 40.026749644748875


En general tenemos medias bastante parecedidas pero tenemos un problema con el error cuadratico medio en la primera y tercera región.

# Cálculo de ganancias

In [12]:
def Ganancias (rmse, mean_pred,threshold_units=111.1):
    
    print(f"El volumen medio de reservas es de {mean_pred:.2f} unidades.")
    print(f"Para evitar pérdidas, cada pozo debe producir al menos {threshold_units:.2f} unidades en promedio.") 
     
    if mean_pred >= threshold_units:
         print('La cantidad media de reservas es suficiente para evitar pérdidas, es probable que la inversión sea rentable')
    else:
         print('La cantidad media de reservas no es suficiente para evitar pérdidas, es probable que la inversión no sea rentable.')
         


In [13]:
# Ganancias de la region 0
Ganancias (rmse_0, mean_pred_0)

El volumen medio de reservas es de 92.47 unidades.
Para evitar pérdidas, cada pozo debe producir al menos 111.10 unidades en promedio.
La cantidad media de reservas no es suficiente para evitar pérdidas, es probable que la inversión no sea rentable.


In [14]:
# Ganancias de la region 1
Ganancias (rmse_1, mean_pred_1)

El volumen medio de reservas es de 68.73 unidades.
Para evitar pérdidas, cada pozo debe producir al menos 111.10 unidades en promedio.
La cantidad media de reservas no es suficiente para evitar pérdidas, es probable que la inversión no sea rentable.


In [15]:
# Ganancias de la region 2
Ganancias (rmse_2, mean_pred_2)

El volumen medio de reservas es de 94.95 unidades.
Para evitar pérdidas, cada pozo debe producir al menos 111.10 unidades en promedio.
La cantidad media de reservas no es suficiente para evitar pérdidas, es probable que la inversión no sea rentable.


Basandonos en los resultados podemos observar que si elegimos aleatoriamente 200 pozos en cualquiera de las zonas, la media no supera el umbral de la meta que requiere la empresa petrolera para poder generar ganancias, por lo tanto se debe de realizar otras pruebas para elegir correctamente la zona.

# función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones

In [16]:
def top_200_pozos (predictions, target, revenue=4500, investment=100000000):
    data = pd.DataFrame({'predictions': predictions, 'target': target})
    top_200 = data.nlargest(200, 'predictions')
    total_revenue = top_200['target'].sum() * revenue
    ganancia = total_revenue - investment 
    print('Ganancia potencial para la región es :', ganancia)
    return top_200, ganancia

In [17]:
top_200_0, ganancia_0 = top_200_pozos (predictions_0, target_valid_0)

Ganancia potencial para la región es : 33208260.43139851


In [18]:
top_200_1, ganancia_1 = top_200_pozos (predictions_1, target_valid_1)

Ganancia potencial para la región es : 24150866.966815114


In [19]:
top_200_2, ganancia_2 = top_200_pozos (predictions_2, target_valid_2)

Ganancia potencial para la región es : 27323911.4313343


Como se puede observar la mejor region para invertir es la 0 con una ganancia posible de 33208260.43139851

# Calcula riesgos y ganancias para cada región

In [20]:
def top_200(predictions, target):
    # Convertir a arrays de numpy en caso de que no lo sean
    target = np.asarray(target)
    predictions = np.asarray(predictions)
    
    sorted_indices = np.argsort(predictions)[::-1]
    top_200_indices = sorted_indices[:200]
    selected_predictions = predictions[top_200_indices]
    selected_target = target[top_200_indices]
    
    return selected_predictions, selected_target 

def bootstrap_profit(predictions, target, n_iterations=1000, sample_size=500, investment=100000000, revenue=4500):
    state = np.random.RandomState(12345)
    ganancias = []
    for i in range(n_iterations):
        # Tomar una muestra bootstrap de las predicciones y target
        sample_predictions, sample_target = resample(predictions, target, n_samples=sample_size, random_state=state)
        
        # Seleccionamos los 200 mejores pozos de las muestras
        _, sample_top_target = top_200(sample_predictions, sample_target)
        
        # Calculamos el beneficio total para esta muestra
        total_revenue = sample_top_target.sum() * revenue
        ganancia = total_revenue - investment
        ganancias.append(ganancia)
        
       
    # Convertir los beneficios a un array de numpy
    ganancias = np.array(ganancias)    
    # Calcular el beneficio promedio
    avg_ganancias = ganancias.mean()   
    # Calcular el intervalo de confianza del 95%
    conf_interval = np.percentile(ganancias, [2.5, 97.5])   
    # Calcular el riesgo de pérdidas (beneficio negativo)
    risk_of_loss = (ganancias < 0).mean() * 100  
    
    return avg_ganancias, conf_interval, risk_of_loss

In [21]:
avg_ganancias_0, conf_interval_0, risk_of_loss_0 = bootstrap_profit(predictions_0, target_valid_0)
avg_ganancias_1, conf_interval_1, risk_of_loss_1 = bootstrap_profit(predictions_1, target_valid_1)
avg_ganancias_2, conf_interval_2, risk_of_loss_2 = bootstrap_profit(predictions_2, target_valid_2)
print("Región 0: Beneficio promedio =", avg_ganancias_0, ", IC 95% =", conf_interval_0, ", Riesgo de pérdidas =", risk_of_loss_0, "%")
print("Región 1: Beneficio promedio =", avg_ganancias_1, ", IC 95% =", conf_interval_1, ", Riesgo de pérdidas =", risk_of_loss_1, "%")
print("Región 2: Beneficio promedio =", avg_ganancias_2, ", IC 95% =", conf_interval_2, ", Riesgo de pérdidas =", risk_of_loss_2, "%")

Región 0: Beneficio promedio = 3964927.666096237 , IC 95% = [-1088816.18926101  9138435.96291906] , Riesgo de pérdidas = 6.4 %
Región 1: Beneficio promedio = 4560451.057866608 , IC 95% = [ 338205.09398985 8522894.53866035] , Riesgo de pérdidas = 1.5 %
Región 2: Beneficio promedio = 4053867.734904668 , IC 95% = [-1680474.36516634  9519024.87781419] , Riesgo de pérdidas = 7.8 %


Realizando la tecnica de Bootstrapping con 1000 iteraciones con 500 pozos y de esos 500 pozos se escogen las 200 mejores predicciones para hacer el calculo del beneficio, la Región 1 es la que menor riesgo tiene con un 1.5% de riesgos de perdida. Además podemos observar que la región 1 esta dentro del intervalo de confianza, lo que da un punto extra, además es el unico intervalo que no tiene como banda baja perdidas IC 95% = [ 338205.09398985 8522894.53866035].

# Conclusión
Se cargaron 3 dataframe que tiene un 5 columnas y 100000 registros, no se detectaron problemas con la data. 

Se crearo una funcion que realizaba un modelo de regresion lineal para predecir las reservas de los pozos, apartir de tres variables, en general las medias son parecidas entre la predicion y los valores reales pero el error cuadratico medio es alto.

Al calcular la ganancia nos percatamos que si elegimos aleatoriamente los pozos ninguna region seria rentable por lo que se hizo la seleccion del top 200 de cada region y ha primera vista la region 0 era la mejor opcion con una ganancia posible de 33208260.4313.

Para estar seguro se utilizo la tecnica de Bootstrapping con 1000 iteraciones con 500 pozos para escoger el top 200 tambien se calculo el riesgo de perdida con un intervalo de confianza del 95% para cada region.

Se concluyo que la mejor zona, es la region 1 ya que fue la unica que estuvo dentro del intervalo de confianza y obtuvo una ganancia esperada de [ 338205.09398985 - 8522894.53866035].